## Image classification

Za domácí úkol budete mít vytvořit si vlastní libovolný dataset na image classfication task, abyste si vyzkoušeli, že vyrábět data není úplně snadné. Dataset by měl mít aspoň 5 různých tříd ke klasifikaci a ke každé nějaké rozumné desítky obrázků.

Až budete mít data, tak si na nich zkuste image classification task. Jak už jsem zmínila výše, trénovat celou konvoluční síť od začátku by bylo početně náročné, proto budete mít za úkol si vybrat 3 z už predtrénovaných modelů například [odsud](https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/) a přetrénovat je tak, aby uměly klasifikovat vámi vytvořená data.

Můžete si na vašich datech i zkusit natrénovat podobnou (klidně i stejnou) jednoduchou konvoluční síť, kterou jsme používali před na MNISTu výše a schválně se podívat, jak moc bude špatná oproti těm už předtrénovaným.

Své řešení mi pošlete emailem do stanoveného deadline. Pro sepsání popisu vašeho datasetu, zvolených modelů a rozboru výsledků použijte [tento template](https://github.com/kackamac/Prirodou-inspirovane-algoritmy/blob/master/08_konvoluce/DU2_konvolucni_site.pdf). Řešení by mělo obsahovat:
1. slovní popis vašich dat (jaké máte labely, kolik každý obsahuje obrázků, velikost obrázků, zdůvodnění volby datasetu)
2. váš kód
3. slovní popis výsledků a porovnání modelů (jaké jste použili, jak byl který úspěšný (accuraccy všech modelů na testovací množině, sepsaná třeba v nějaké pěkné tabulce) a který byl nejlepší)
4. vámi vytvořený předzpracovaný dataset (obrázky by měly být malé, takže by neměl být problém s posláním)

Pár tipů
- bylo by dobré mít u všech tříd stejný počet obrázků
- obrázky bude potřeba převést do nějakého jednotného rozumně malého rozměru (například 128 x 128 pixelů)
- když budete používat předtrénované sítě, tak ty vaše obrázky by měly být stejně velké jako ty, na kterých byla trénovaná ta  síť původně
- zároveň bude potřeba je rozdělit na dvě disjunktní množiny -- trénovací a testovací (trénovací na přetrénovaní modelů a testovací na jejich vyhodnocení)
- model se dá přetrénovat například tak, že si nahradíte poslední nebo několik posledních vrstev vlastními, které budete přetrénovávat podle vašich dat
- pokud se vám model nedaří přetrénovat, aby dosahoval nějakých rozumných výsledků, zkuste použít nějakou augmentaci dat a trénovat na větších datech


### Solution


In [ ]:
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras
%matplotlib inline
from tensorflow.keras.layers import Flatten, Dense, Activation, Resizing
from tensorflow.keras.activations import linear, relu
from tensorflow.nn import softmax

In [ ]:
import keras.applications.resnet as resnet
import keras.applications.mobilenet as mobilenet
import keras.applications.xception as xception

In [ ]:
def load_dataset(path, shape=(224,224), test_split=0.2):
    return tf.keras.utils.image_dataset_from_directory(
        path,
        seed=42,
        batch_size=None,
        image_size=shape,
        validation_split=test_split,
        subset="both",
    )

In [ ]:
def preprocess_image(image, network):
    if network == "ResNet50":
        return resnet.preprocess_input(image)
    elif network == "MobileNet":
        return mobilenet.preprocess_input(image)
    elif network == "Xception":
        return xception.preprocess_input(image)
    else:
        raise ValueError("Unknown network")

In [ ]:
def preprocess_input(dataset, network):
    data = []
    target = []

    for image, label in dataset:
        data.append(preprocess_image(image, network))
        target.append(label)
        
    return np.array(data), np.array(target)

In [ ]:
def create_model(base_model, classes):
    base_model.trainable = False
    
    model = keras.models.Sequential([])
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(units=classes, activation=linear, name='logits'))
    model.add(Activation(activation=softmax))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def retrian_model(model, train_data, train_target , epochs=3, batch_size = 20):
    history = model.fit(train_data, train_target, epochs=epochs, validation_split=0.2, batch_size=batch_size).history
    return model, history

In [ ]:
def evaluate_data(model, data, target):
    loss, accuracy = model.evaluate(data, target)
    return accuracy

### Training Models

In [30]:
networks = {
    'ResNet50': resnet.ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3)),
    'MobileNet': mobilenet.MobileNet(include_top=False, weights="imagenet", input_shape=(224, 224, 3)),
    'Xception': xception.Xception(include_top=False, weights="imagenet", input_shape=(299, 299, 3))
}
histories = {}
models = {}

for network in networks.keys():
    print(f"***** Retraining {network} model ***** \n")

    base_model = networks[network]
    train_ds, test_ds = load_dataset("dataset", shape=base_model.input_shape[1:3], test_split=0.2)
    
    train_data, train_target = preprocess_input(train_ds, network)
    test_data, test_target = preprocess_input(test_ds, network)

    classes = len(train_ds.class_names)
    model = create_model(base_model, classes)
    model, history = retrian_model(model, train_data, train_target, 3, 20)

    models[network] = model
    histories[network] = history

    print("\n *** evaluating train data *** ")
    _, train_acc = model.evaluate(train_data, train_target)
    print(f"\n -  Train data accuracy: {train_acc*100:.2f}%")

    print("\n *** evaluating test data *** ")
    _, test_acc = model.evaluate(test_data, test_target)
    print(f"\n- Test data accuracy: {test_acc*100:.2f}%" + "\n")

***** Retraining ResNet50 model ***** 

Found 191 files belonging to 5 classes.
Using 153 files for training.
Using 38 files for validation.
Epoch 1/3
7/7 [==============================] - 75s 8s/step - loss: 1.1372 - accuracy: 0.8115 - val_loss: 0.2789 - val_accuracy: 0.9355
Epoch 2/3
7/7 [==============================] - 40s 6s/step - loss: 2.1055e-05 - accuracy: 1.0000 - val_loss: 0.4578 - val_accuracy: 0.9355
Epoch 3/3
7/7 [==============================] - 31s 5s/step - loss: 6.9862e-07 - accuracy: 1.0000 - val_loss: 0.6959 - val_accuracy: 0.9032

 *** evaluateing train data ***
 
5/5 [==============================] - 25s 4s/step - loss: 0.1410 - accuracy: 0.9804
 -  Train data accuracy: 98.04% ***

 *** evaluateing test data ***
 
2/2 [==============================] - 6s 905ms/step - loss: 0.3007 - accuracy: 0.9737
- Test data accuracy: 97.37% ***

***** Retraining MobileNet model ***** 

Found 191 files belonging to 5 classes.
Using 153 files for training.
Using 38 files for

: 

: 

##### Show plots

In [ ]:
for network in networks.keys():
    plt.plot(histories[network]['accuracy'], label="Train data", color='red')
    plt.plot(histories[network]['val_accuracy'], label="Valitation data", color='blue')

    plt.title(f'{network} accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
import os
from PIL import Image
label = "airplane"
path = f"dataset\\{label}"

list = os.listdir(path)
for i, image_path in enumerate(list):
    image = Image.open(path + "\\" + image_path)
    image = image.resize((300, 300))
    image.save("dataset\\resized\\" + f"{label}_" +str(i+1) + ".jpg")